O objetivo deste notebook é implementar uma árvore de decisão do zero (numpy e pandas), que possuirá a seguinte API:

```Python
df = pd.read_csv("data.csv")

train_df, test_df = train_test_split(df, test_size=0.2)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)
```

Se der tempo, colocar diagrama do algoritmo

# Impotar bibliotecas

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint

In [ ]:
%matplotlib inline
sns.set_style("darkgrid")

# Carregar e preparar os dados

#### Formato dos dados
- Nesse caso, a última coluna do dataset é o atributo alvo

In [ ]:
df = pd.read_csv("data/Iris.csv")
df = df.drop("Id", axis=1)
df = df.rename(columns={"species": "label"})
df.shape

In [ ]:
df.head()

In [ ]:
df['label'].hist()

# Divisão de treino e teste

Como as classes estão distruídas igualmente, não precisamos realizar a amostragem estratificada

In [ ]:
# @param test_size can be an integer or float (percentage) of the dataset size.
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    # extract all indexes from dataset
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [ ]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

# Funções auxiliares

As funções auxiliares irão trabalhar com NumPy arrays. Logo, iremos atribuir os valores no formato numpy a uma variável

In [ ]:
data = train_df.values
data[:5]

### O conjunto ou subconjunto de dados está puro?

In [ ]:
# Check if there is only one class on target column
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

### Classificação

Definida pela moda em um problema de classificação

In [ ]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

### Potenciais divisões

Utilizada para extrair os possíveis valores para os ramos das árvores

In [ ]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

### Divisão dos dados

Funciona com dados categóricos ou numéricos

In [ ]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        left = data[split_column_values <= split_value]
        right = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        left = data[split_column_values == split_value]
        right = data[split_column_values != split_value]
    
    return left, right

### Information Gain

O cálculo de Information Gain permite selecionar a melhor divisão dos dados

In [ ]:
def calc_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [ ]:
def calc_info_gain_numeric(df, left, right, entropy_before_split, split_values):
    
    total_elements = df.shape[0]
    ent_left = calc_entropy(left)
    
    ent_right = calc_entropy(right)
    
    weighted_entropy = ((left.shape[0] / total_elements) * ent_left) + (
        (right.shape[0] / total_elements) * ent_right
    )
    info_gain = entropy_before_split - weighted_entropy
    
    return info_gain

In [ ]:
def determine_best_split(data, potential_splits):
    
    best_information_gain = -9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_left, data_right = split_data(data, split_column=column_index, split_value=value)
            information_gain = calc_info_gain_numeric(data, data_left, data_right, calculate_entropy(data), value)
            
            if information_gain >= best_information_gain:
                best_information_gain = information_gain
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

# Algoritmo da Árvore de Decisão

### Representação da árvore de decisão

In [ ]:
sub_tree = {"question": ["yes_answer", 
                         "no_answer"]}

In [ ]:
example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
                                      {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
                                                                                        "Iris-virginica"]}, 
                                                                "Iris-virginica"]}]}

### Determinar o tipo de atributo

In [ ]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

### Algoritmo completo

In [ ]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # Data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # Stop condition
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_left, data_right = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_left) == 0 or len(data_right) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_left, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_right, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the question.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

# Classification

In [ ]:
sub_tree

In [ ]:
example = test_df.iloc[0]
example

In [ ]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # leaft node
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [ ]:
classify_example(example, tree)

# Calculate Accuracy

In [ ]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

Train accuracy

In [ ]:
accuracy = calculate_accuracy(train_df, tree)
accuracy

Test accuracy

In [ ]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

In [ ]:
import sklearn.tree as SKTree
from sklearn.metrics import accuracy_score

clf = SKTree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(train_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train_df['label'])

print(f"Train accuracy = {accuracy_score(train_df['label'],  clf.predict( train_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']] ) )}")
print(f"Test accuracy = {accuracy_score(test_df['label'],  clf.predict( test_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']] ) )}")


In [ ]:
pprint(tree, width=50)

In [ ]:
SKTree.plot_tree(clf) 

In [ ]:
# Exportar árvore gerada pelo sklearn em PDF

import graphviz 
dot_data = SKTree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris") 